## Finding Missing Requirements

In this notebook, I try to put together an iterative way to:
1. Add *unbound variables* until the requirement is satisfied
2. Get the *relationships* of the unbound variables
3. Filter these relationships by adding one at a time as a negated constraint
   and check again the model.
4. When we find the one that makes the model unsat, it means we've found the right one.

#### Goals
- Make the search of finding the correct relationship as fast as possible:
  usually relationships are quite a lot, and we need to iterate them at least a
  couple of times. We can add progressively only the relationships that remain
  after each pass, always one at a time and checking the model again.

In [649]:
from mc_openapi.doml_mc.intermediate_model.metamodel import parse_metamodel, parse_inverse_associations
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_doml_model
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_xmi_model
from mc_openapi.doml_mc import DOMLVersion
from z3 import Solver, Model, DatatypeSortRef, FuncDeclRef, Consts, ForAll, Exists, Implies, And, Or, Not, sat, unsat
import yaml

from mc_openapi.doml_mc.z3encoding.metamodel_encoding import (
    def_association_rel,
    def_attribute_rel,
    mk_association_sort_dict,
    mk_attribute_sort_dict,
    mk_class_sort_dict
)
from mc_openapi.doml_mc.z3encoding.im_encoding import (
    assert_im_associations_q,
    assert_im_attributes,
    def_elem_class_f_and_assert_classes,
    mk_elem_sort_dict,
    mk_stringsym_sort_dict
)
from mc_openapi.doml_mc.z3encoding.types import Refs
from mc_openapi.doml_mc.z3encoding.utils import mk_attr_data_sort

from itertools import product
from operator import itemgetter

In [650]:
with open("../assets/doml_meta_v2.0.yaml") as mmf:
    mmdoc = yaml.load(mmf, yaml.Loader)
mm = parse_metamodel(mmdoc)
inv_assoc = parse_inverse_associations(mmdoc)

**You can change here the input DOML file**

In [651]:
# doml_document_path = "../../tests/doml/nginx-openstack_v2.0.domlx"
doml_document_path = "../../tests/doml/nginx-openstack_v2.0_wrong_vm_iface.domlx"
# doml_document_path = "../../tests/doml/nginx-openstack_v2.0_wrong_iface_uniq.domlx"

In [652]:
with open(doml_document_path, "rb") as xmif:
    doc = xmif.read()

im, _ = parse_doml_model(doc, DOMLVersion.V2_0)
doml_xmi = parse_xmi_model(doc, DOMLVersion.V2_0)

We need to initialize each time the Solver context before iterating,
since an unbound variable is an element, and elements are an EnumSort,
and EnumSorts cannot be modified and depend on the solver context.

The following code is stuff that is already present in the `IntermediateModelChecker`.

In [653]:
from typing import Dict

Context = Dict

In [654]:
def initialize_solver(
    unbound_elems_quantity: int = 0,
    requirements: list = []
) -> Context:
    ctx = dict()
    
    ctx["solver"] = Solver()

    ctx["class_sort"], ctx["class_"] = mk_class_sort_dict(mm, ctx["solver"].ctx)
    ctx["assoc_sort"], ctx["assoc"] = mk_association_sort_dict(mm, ctx["solver"].ctx)
    ctx["attr_sort"], ctx["attr"] = mk_attribute_sort_dict(mm, ctx["solver"].ctx)
    ctx["ss_sort"], ctx["ss"] = mk_stringsym_sort_dict(im, mm, ctx["solver"].ctx)
    ctx["attr_data"] = mk_attr_data_sort(ctx["ss_sort"], ctx["solver"].ctx)

    ctx["unbound_elems"] = [f"unbound{i}" for i in range(unbound_elems_quantity)]
    ctx["elem_sort"], ctx["elem"] = mk_elem_sort_dict(im, ctx["solver"].ctx, ctx["unbound_elems"])

    ctx["elem_class_f"] = def_elem_class_f_and_assert_classes(
        im,
        ctx["solver"],
        ctx["elem_sort"],
        ctx["elem"],
        ctx["class_sort"],
        ctx["class_"]
    )
    ctx["attr_rel"] = def_attribute_rel(
        ctx["attr_sort"],
        ctx["elem_sort"],
        ctx["attr_data"]
    )
    assert_im_attributes(
        ctx["attr_rel"],
        ctx["solver"],
        im,
        mm,
        ctx["elem"],
        ctx["attr_sort"],
        ctx["attr"],
        ctx["attr_data"],
        ctx["ss"]
    )
    ctx["assoc_rel"] = def_association_rel(
        ctx["assoc_sort"],
        ctx["elem_sort"]
    )
    assert_im_associations_q(
        ctx["assoc_rel"],
        ctx["solver"],
        {k: v for k, v in im.items() if k not in ctx["unbound_elems"]},
        ctx["elem"],
        ctx["assoc_sort"],
        ctx["assoc"],
    )

    # Add requirements
    for req in requirements:
        req(ctx)

    return ctx

In [655]:
def requirement_1(ctx: Context):    
    vm, iface = Consts(
        "vm iface", ctx["elem_sort"]
    )
    vmIfaceAssertion = ForAll(
        [vm],
        Implies(
            ctx["elem_class_f"](vm) == ctx["class_"]["infrastructure_VirtualMachine"],
            Exists(
                [iface],
                And(
                    ctx["assoc_rel"](vm, ctx["assoc"]["infrastructure_ComputingNode::ifaces"], iface)
                )
            )
        )
    )
    ctx["solver"].assert_and_track(vmIfaceAssertion, "vm_iface")

In [656]:
def do_some_checking(ubvars_n: int = 0) -> Context:
    ctx = initialize_solver(ubvars_n, [requirement_1])
    solver = ctx["solver"]

    res = solver.check()

    if res == sat:
        return ctx
    elif res == unsat:
        # print(f"ubvars={ubvars_n}; UNSAT_CORE:")
        # print(solver.unsat_core())
        print(f"Increasing unbound vars to {ubvars_n + 1}")
        return do_some_checking(ubvars_n + 1)
    else:
        raise RuntimeError("It took too long to decide.")

In [657]:
solved_ctx = do_some_checking()
solved_model = solved_ctx["solver"].model()

Increasing unbound vars to 1


In [658]:
def get_ubvars_and_assoc(ctx: Context, model: Model):
    elem, assoc, assoc_rel, unbound_elems = itemgetter("elem", "assoc", "assoc_rel", "unbound_elems")(ctx)

    return [ ((e1n, e1), a, (e2n, e2)) 
        for (e1n, e1), a, (e2n, e2) in product(elem.items(), assoc.values(), elem.items()) 
        if (e1n in unbound_elems or e2n in unbound_elems) and model.eval(assoc_rel(e1, a, e2))
    ]

In [659]:
def pretty_ubvar_assoc(assoc):
    (e1n, e1), a, (e2n, e2) = assoc
    tokens = str(str(e1) + " " + str(a) + " " + str(e2)).split()
    ret_str = ""
    for token in tokens:
        if token[0:4] == "elem":
            value = im.get(token)
            ret_str += f"{value.class_} ({value.user_friendly_name})" if value else f"<'{token}' not found>"
        else:
            ret_str += token
        ret_str += " "
    return ret_str.strip()

As we can see, there are many associations that involve the unbound variable `unbound0`.
We can see that between them, there is the one we really want:

`infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::ifaces unbound0`

We now need to filter out all the others, by taking one of these lines at a time, and adding it as a negated constraint.

NOTE: It appears that the list is not deterministic. Sometimes at the end there are associations that start from `unbound0`, sometimes there is none. 

In [660]:
ubvars_and_assoc = get_ubvars_and_assoc(solved_ctx, solved_model)
# print("\n".join([pretty_ubvar_assoc(assoc) for assoc in ubvars_and_assoc]))

In [661]:
# def thin_ubvars_and_assoc(ctx: Context, ubvars_and_assoc):
#     (e1_name, e1), a, (e2_name, e2) = ubvars_and_assoc[0]

#     ctx["solver"].assert_and_track(Not(ctx["assoc_rel"](e1, a, e2)), f"__neg_{e1_name}_{str(a)}_{e2_name}")
    
#     res = ctx["solver"].check()
#     if res == sat:
#         print("Trying again with one more constraint")
#         # Get new ubvars_and_assoc
#         model = ctx["solver"].model()
#         thinned_ubvars_and_assoc = get_ubvars_and_assoc(ctx, model)
        
#         # Print table showing the diff
#         from difflib import context_diff
#         uvar_as_text = lambda input: [pretty_ubvar_assoc(assoc) for assoc in input]
#         print("\n".join([a for a in context_diff(uvar_as_text(ubvars_and_assoc), uvar_as_text(thinned_ubvars_and_assoc))]))

#         # Iterate
#         thin_ubvars_and_assoc(ctx, thinned_ubvars_and_assoc)
#     else:
#         print("Last constraint was the right association!")

# thin_ubvars_and_assoc(solved_ctx, ubvars_and_assoc)

In [662]:
def thin_ubvars_and_assoc(ctx: Context, ubvars_and_assoc: list):
    """Returns a tuple where the first item is
    """
    if not ubvars_and_assoc:
        return []


    (e1_name, e1), a, (e2_name, e2) = assoc = ubvars_and_assoc[0]
    assoc_rel = ctx["assoc_rel"](e1, a, e2)
    
    print("-----------------------------------------")

    # Add negated constraint
    print(f"\tAdd constraint Not({pretty_ubvar_assoc(ubvars_and_assoc[0])})")
    ctx["solver"].push()
    # ctx["solver"].assert_and_track(Not(assoc_rel), f"__neg_{e1_name}_{str(a)}_{e2_name}")
    ctx["solver"].add(Not(assoc_rel))
    
    res = ctx["solver"].check()
    if res == sat:
        print("SAT:\tAdding one more constraint and trying again")
        # Get new ubvars_and_assoc
        model = ctx["solver"].model()
        thinned_ubvars_and_assoc = get_ubvars_and_assoc(ctx, model)
        
        # Print table showing the diff
        from difflib import context_diff
        uvar_as_text = lambda input: [pretty_ubvar_assoc(assoc) for assoc in input]
        print("\n".join([a for a in context_diff(uvar_as_text(ubvars_and_assoc), uvar_as_text(thinned_ubvars_and_assoc), lineterm="", fromfile='Before', tofile="After")]))

        # Iterate
        return thin_ubvars_and_assoc(ctx, thinned_ubvars_and_assoc)
    else:
        print("UNSAT\tLast constraint was the association we are looking for!")
        ctx["solver"].pop()
        
        if ubvars_and_assoc[1:]:
            print("\tIterating over")
            print("\t\t" + "\n\t\t".join([pretty_ubvar_assoc(assoc) for assoc in ubvars_and_assoc[1:]]))
        return [*set([assoc] + thin_ubvars_and_assoc(ctx, ubvars_and_assoc[1:]))]


assoc_to_implement = thin_ubvars_and_assoc(solved_ctx, ubvars_and_assoc)

-----------------------------------------
	Add constraint Not(infrastructure_VirtualMachine (vm1) commons_Property::reference unbound0)
SAT:	Adding one more constraint and trying again
*** Before
--- After
***************
*** 1,37 ****
- infrastructure_VirtualMachine (vm1) commons_Property::reference unbound0
- infrastructure_VirtualMachine (vm1) application_SoftwareComponent::exposedInterfaces unbound0
- infrastructure_VirtualMachine (vm1) application_SoftwareComponent::consumedInterfaces unbound0
- infrastructure_VirtualMachine (vm1) application_SaaS::exposedInterfaces unbound0
- infrastructure_VirtualMachine (vm1) infrastructure_InfrastructureLayer::nodes unbound0
- infrastructure_VirtualMachine (vm1) infrastructure_InfrastructureLayer::storages unbound0
- infrastructure_VirtualMachine (vm1) infrastructure_InfrastructureLayer::faas unbound0
  infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::ifaces unbound0
- infrastructure_VirtualMachine (vm1) infrastructure_Computi

In [663]:
print("\n".join([pretty_ubvar_assoc(assoc) for assoc in assoc_to_implement]))

infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::ifaces unbound0


From here, we should then add this in the Intermediate Model, and then in the ECore (?) to generate the DOML file somehow.

We can then provide the new file, or a diff to be patched onto the original?